In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark

In [2]:
import os
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

findspark.init()

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [4]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

config = SparkConf().setMaster('local').setAppName('lab21')
spark = SparkSession.builder.config(conf=config).getOrCreate()
sc = spark.sparkContext

In [12]:
DATASET_PATH = '/content/gdrive/MyDrive/fakefriends.csv'
lines = sc.textFile(DATASET_PATH)

def parseLine(line):
  fields = line.split(',')
  age = int(fields[2])
  num_friends = int(fields[3])
  return (age, num_friends)

rdd = lines.map(parseLine)

In [ ]:
totalsByAge = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
averageByAge = totalsByAge.mapValues(lambda x: round(x[0]/ x[1], 2))
for result in averageByAge.collect():
  print(result)